In [ ]:
# C12. Programmer les tests automatisés d’un modèle d’intelligence artificielle en définissant les règles devalidation des jeux de données, des étapes de
# préparation des données, d'entraînement, d’évaluation et de validation du modèle pour permettre son intégration en continu 
# et garantir un niveau de qualité élevé.


In [ ]:
# 1- Cas de Test et Règles de Validation par Étape

#     Préparation des Données
#         Partie visée : Traitement et validation des données d’entrée avant calcul des embeddings.
#         Règles de validation :
#             Vérifier que les données textuelles sont au format attendu (ex : chaînes non nulles, sans caractères spéciaux inappropriés).
#             S'assurer qu'il n'y a pas de valeurs manquantes ou nulles dans les données textuelles à encoder.
#         Stratégie : Exécuter des tests sur un échantillon de données pour confirmer que les textes sont dans un format compatible avec le modèle d'embeddings (SentenceTransformer).

#     Calcul des Embeddings
#         Partie visée : Fonction de calcul des embeddings.
#         Règles de validation :
#             Les embeddings produits doivent avoir une dimension constante (ici, 384 pour all-MiniLM-L6-v2).
#             Aucun embedding ne doit contenir de valeurs infinies ou NaN.
#         Stratégie : Vérifier la taille des embeddings générés sur un échantillon de données et confirmer qu'ils ne contiennent pas de valeurs problématiques.

#     Calcul de la Similarité Cosinus
#         Partie visée : Fonction calculate_cosine_similarity_in_batches.
#         Règles de validation :
#             La matrice de similarité doit être symétrique (ce qui est attendu pour une similarité cosinus).
#             La taille de la matrice de similarité doit être correcte (dimensions N x N pour N embeddings).
#         Stratégie : Utiliser des embeddings factices et vérifier la symétrie ainsi que les dimensions de la matrice.

#     Recommandation de Livres (Évaluation du Modèle)
#         Partie visée : Fonction recommander_livres_sans_categorie.
#         Règles de validation :
#             La fonction doit exclure le livre de référence des résultats de recommandation.
#             Les livres recommandés doivent être triés par ordre décroissant de similarité.
#             Au moins 3 recommandations doivent être fournies (si disponible).
#         Stratégie : Utiliser un ensemble de données de test contenant plusieurs titres et vérifier que les recommandations respectent ces critères.

In [ ]:
# 2 Les outils de test (framework, bibliothèque, etc.) choisis sont cohérents avec l’environnement technique du projet.

# Pour valider ce point, nous avons sélectionné des outils de test compatibles avec l'environnement de notebook basé sur Python, PyTorch, et SentenceTransformers.
# Ces outils sont adaptés aux tests modulaires et à la validation de chaque étape dans des cellules individuelles.


# Outils de Test Choisis et Cohérence avec l’Environnement
# 1. Framework de Test : Utilisation des Assertions en Notebook

#     Raisons :
#         Nous utilisons des assertions directement dans les cellules de notebook pour vérifier les conditions de test, ce qui est adapté 
#         pour un environnement interactif.
#         Cela permet d'exécuter chaque test indépendamment et d'obtenir des résultats immédiats sans avoir besoin d’un framework externe.
#     Installation : Aucune installation requise pour l’utilisation des assertions.

# 2. Manipulation et Validation de Données : pandas

#     Raisons :
#         pandas est utilisé pour gérer les données de livres (titres, auteurs, etc.) ; 
#         il permet de manipuler et de vérifier facilement les jeux de données pendant les tests en notebook.

# 3. Modélisation et Calculs avec PyTorch : torch

#     Raisons :
#         torch est utilisé pour les embeddings et le calcul de similarité cosinus ; 
#         il est donc cohérent de l’utiliser également dans les tests pour vérifier la structure et le contenu des matrices de similarité.



In [ ]:
3#  Les tests sont intégrés et respectent la couverture souhaitée établie.

# Pour valider ce troisième point, nous allons écrire des cellules de tests en notebook qui couvrent chaque étape du processus 
# (préparation des données, calcul des embeddings, similarité cosinus, et recommandations).
# Ces tests assureront une couverture complète des fonctionnalités critiques pour le modèle de recommandation.




In [ ]:
# 1 Test de Préparation des Données pour Toutes les Colonnes Clés

# Ce test couvre chaque colonne du fichier CSV et vérifie :



    # title et authors :
    #     Vérifie l'absence de valeurs nulles et vides.

    # description :
    #     Assure qu'il n'y a pas de valeurs nulles ou vides et que les descriptions sont suffisamment longues.

    # published_year :
    #     Vérifie que la colonne ne contient pas de valeurs nulles et que chaque valeur est un entier (représentant une année valide).

    # average_rating :
    #     Vérifie l'absence de valeurs nulles et s'assure que chaque valeur est un nombre dans la plage de 0 à 5.

    # categories :
    #     Vérifie l'absence de valeurs nulles et confirme que chaque entrée contient du texte.

In [7]:
import pandas as pd

# Chargement du fichier CSV pour vérification
csv_path = r"C:\Users\User\Downloads\CoursAlternance\Chefoeuvre\RecommendationsLectures\final_dataset_clean.csv"
data = pd.read_csv(csv_path)

# Vérification des colonnes essentielles
# 1. Vérification de la colonne 'title'
assert data['title'].notnull().all(), "Certaines entrées de titres sont nulles dans le CSV."
assert data['title'].str.strip().astype(bool).all(), "Certaines entrées de titres sont vides."

# 2. Vérification de la colonne 'authors'
assert data['authors'].notnull().all(), "Certaines entrées d'auteurs sont nulles dans le CSV."
assert data['authors'].str.strip().astype(bool).all(), "Certaines entrées d'auteurs sont vides."

# 3. Vérification de la colonne 'description' avec liste des titres si description < 1 caractère
MIN_DESC_LENGTH = 1
short_descriptions = data[data['description'].str.len() < MIN_DESC_LENGTH]

# Si des descriptions sont trop courtes, lister les titres concernés
if not short_descriptions.empty:
    short_titles = short_descriptions['title'].tolist()
    print(f"Livres avec des descriptions trop courtes (< {MIN_DESC_LENGTH} caractères) :")
    for title in short_titles:
        print(f"- {title}")
else:
    print(f"Toutes les descriptions font au moins {MIN_DESC_LENGTH} caractère(s).")

# 4. Vérification de la colonne 'published_year' pour accepter entiers et floats
assert data['published_year'].notnull().all(), "Certaines années de publication sont nulles dans le CSV."
assert data['published_year'].apply(lambda x: isinstance(x, (int, float))).all(), "Certaines années de publication ne sont ni des entiers ni des floats."

# 5. Vérification de la colonne 'average_rating'
assert data['average_rating'].notnull().all(), "Certaines notes moyennes sont nulles dans le CSV."
assert data['average_rating'].apply(lambda x: isinstance(x, (int, float)) and 0 <= x <= 5).all(), "Certaines notes moyennes ne sont pas dans la plage attendue (0-5)."

# 6. Vérification de la colonne 'categories'
assert data['categories'].notnull().all(), "Certaines catégories sont nulles dans le CSV."
assert data['categories'].str.strip().astype(bool).all(), "Certaines entrées de catégories sont vides."

print("Test de préparation des données pour toutes les colonnes clés : OK")


Toutes les descriptions font au moins 1 caractère(s).
Test de préparation des données pour toutes les colonnes clés : OK


In [ ]:
# 2 Test de Calcul des Embeddings

# Ce test vérifie que la colonne description est correctement transformée en embeddings, 
# avec la bonne dimension (384 pour le modèle all-MiniLM-L6-v2) et sans valeurs infinies ou NaN.

In [14]:
from sentence_transformers import SentenceTransformer
import torch

# Charger le modèle SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

# Générer des embeddings pour les 30 premières descriptions
example_descriptions = data['description'].iloc[:30].tolist()
embeddings = model.encode(example_descriptions, convert_to_tensor=True)

# Test : Vérifier la dimension des embeddings
assert embeddings.shape == (30, 384), "La dimension des embeddings n'est pas correcte."

# Test : Vérifier l'absence de NaN ou de valeurs infinies
assert torch.isfinite(embeddings).all(), "Les embeddings contiennent des valeurs infinies ou NaN."
print("Test de calcul des embeddings : OK")


Test de calcul des embeddings : OK


In [ ]:
# 3 Test du Calcul de Similarité Cosinus en Lots

# Ce test vérifie la symétrie de la matrice de similarité cosinus et ses dimensions. 
# La fonction utilise le produit matriciel pour calculer les similarités en lots.

In [15]:
# Fonction pour calculer la similarité cosinus par lots
def calculate_cosine_similarity_in_batches(embeddings, batch_size=10):
    cosine_sim_list = []
    for i in range(0, embeddings.shape[0], batch_size):
        batch_embeddings = embeddings[i:i + batch_size]
        batch_cosine_sim = torch.mm(batch_embeddings, embeddings.T)  # Produit matriciel
        cosine_sim_list.append(batch_cosine_sim)
    return torch.cat(cosine_sim_list)

# Calculer la similarité cosinus pour les 30 descriptions
cosine_sim = calculate_cosine_similarity_in_batches(embeddings)

# Test : Vérifier les dimensions de la matrice de similarité
assert cosine_sim.shape == (30, 30), "La matrice de similarité cosinus a des dimensions incorrectes."

# Test : Vérifier la symétrie de la matrice de similarité
assert torch.allclose(cosine_sim, cosine_sim.T, atol=1e-6), "La matrice de similarité cosinus n'est pas symétrique."
print("Test du calcul de similarité cosinus : OK")


Test du calcul de similarité cosinus : OK


In [17]:
# Afficher les 30 premiers titres du DataFrame
subset_data = data.iloc[:30]  # Sous-ensemble limité aux 30 premières lignes

print("Les 30 premiers titres du dataset :")
for idx, title in enumerate(subset_data['title'], start=1):
    print(f"{idx}. {title}")


Les 30 premiers titres du dataset :
1. Gilead
2. Spider's Web
3. The One Tree
4. Rage of angels
5. The Four Loves
6. The Problem of Pain
7. An Autobiography
8. Empires of the Monsoon
9. The Gap Into Madness
10. Master of the Game
11. If Tomorrow Comes
12. Assassin's Apprentice
13. Warhost of Vastmark
14. The Once and Future King
15. Murder in LaMut
16. Jimmy the Hand
17. Witness for the Prosecution & Selected Plays
18. The Little House
19. Mystical Paths
20. Glittering Images
21. Glamorous Powers
22. The Mad Ship
23. Post Captain
24. The Reverse of the Medal
25. Miss Marple
26. The Years of Rice and Salt
27. Spares
28. Gravity
29. The Wise Woman
30. Girls' Night in


In [ ]:
# 4 Test de la Fonction de Recommandation

# Ce test vérifie que la fonction de recommandation retourne les livres les plus similaires 
# sans inclure le livre recherché lui-même dans les résultats.

In [18]:
# Limiter le DataFrame aux 30 premières lignes pour ce test
subset_data = data.iloc[:30]

# Fonction de recommandation modifiée pour travailler avec le sous-ensemble
def recommander_livres_sans_categorie(titre_livre, data=subset_data, cosine_sim=cosine_sim, nb_recos=3):
    # Chercher le livre par son titre (insensible à la casse)
    results = data[data['title'].str.contains(titre_livre, case=False, na=False)]
    
    # Vérification de l'existence du livre
    if results.empty:
        print(f"Le livre '{titre_livre}' n'existe pas dans la base.")
        return []
    
    # Récupération de l'index du premier résultat correspondant
    idx = results.index[0]

    # Calcul des scores de similarité pour le livre recherché
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Trier les livres par similarité (du plus similaire au moins similaire)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Créer la liste des recommandations en excluant le livre lui-même (le premier de la liste)
    recommendations = []
    for i, score in sim_scores[1:nb_recos+1]:  # Ignorer le premier (le même livre)
        livre_info = data.iloc[i]
        recommendations.append((livre_info['title'], livre_info['authors'], score.item()))
    
    return recommendations

# Exécuter le test de recommandation sur le sous-ensemble de données
titre_recherche = "Gilead"
recommendations = recommander_livres_sans_categorie(titre_recherche)

# Test : Vérifier que le livre recherché est exclu des résultats
assert all(rec[0] != titre_recherche for rec in recommendations), "Le livre recherché est inclus dans les recommandations."

# Test : Vérifier le nombre de recommandations (3 ou moins si moins de livres similaires sont disponibles)
assert len(recommendations) == min(3, len(subset_data) - 1), f"Le nombre de recommandations est incorrect : {len(recommendations)}"

print("Test de la fonction de recommandation : OK")
print("Recommandations :", recommendations)


Test de la fonction de recommandation : OK
Recommandations : [('Post Captain', "Patrick O'Brian", 0.3553885817527771), ('The Little House', 'Philippa Gregory', 0.3518153429031372), ('The One Tree', 'Stephen R. Donaldson', 0.3289688527584076)]


In [ ]:
# Documentation pour l’Installation et l’Exécution des Tests
# 1. Installation de l’Environnement de Test

#     Environnement requis :
#         Python 3.8 ou supérieur
#         Accès à internet pour l’installation des packages

# 2. Dépendances Requises

# Voici les principales dépendances listées également dans le fichier requirements.txt :

#     pandas : Manipulation des données pour les livres
#     torch : Calcul des embeddings et similarités cosinus
#     sentence-transformers : Modèle de transformation all-MiniLM-L6-v2
#     pytest et pytest-cov : Outils pour les tests et la couverture de code

# 3. Exécution des Tests

#     Lancer les tests unitaires :
#         Les tests sont configurés dans des cellules de notebook,un par un donc pour bien visualiser .